# Lab 5: Radiation in a Global Climate Model

The goals of this lab are to practice thinking like a climate modeller, and interpreting figures.

Our science questions today are:
- How do specific greenhouse gases affect the temperature profile of the atmosphere?
- How does vertical motion in the atmosphere affect the radiative balance?

In this lab, we will develop some tools that help us address that question quantitatively.

We will build on our experience with the theoretical N-layer atmosphere from previous weeks.  We will plot and interpret **skew-T diagrams** for the atmosphere's vertical structure, use a more sophisticated **radiative transfer model**, and compute **instantaneous radiative forcing**.

***
Import data and helper functions to make skew-T diagrams throughout the lab:

In [ ]:
#  This code is used just to create the skew-T plot of global, annual mean air temperature
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from metpy.plots import SkewT  ## using a package called MetPy for skew-T functionality

ncep_url = "http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis.derived/"
ncep_air = xr.open_dataset( ncep_url + "pressure/air.mon.1981-2010.ltm.nc", use_cftime=True)
#  Take global, annual average 
coslat = np.cos(np.deg2rad(ncep_air.lat))
weight = coslat / coslat.mean(dim='lat')
Tglobal = (ncep_air.air * weight).mean(dim=('lat','lon','time'))

In [ ]:
def make_skewT():
    fig = plt.figure(figsize=(9, 9))
    skew = SkewT(fig, rotation=30)
    skew.plot(Tglobal.level, Tglobal, color='black', linestyle='-', linewidth=2, label='Observations')
    skew.ax.set_ylim(1050, 10)
    skew.ax.set_xlim(-90, 45)
    # Add the relevant special lines
    skew.plot_dry_adiabats(linewidth=0.5)
    skew.plot_moist_adiabats(linewidth=0.5)
    #skew.plot_mixing_lines()
    skew.ax.legend()
    skew.ax.set_xlabel('Temperature (degC)', fontsize=14)
    skew.ax.set_ylabel('Pressure (hPa)', fontsize=14)
    return skew

In [ ]:
skew = make_skewT()

---
## Demonstration and skills practice
You already reviewed some of your coding skills with the "Function Fix-It" exercises: defining named variables, functions, and default values.  I will demonstrate how to set up a Climlab instance with radiative transfer, and then with radiative-convective modules enabled.

---
### 1. Setting up a radiative transfer model

We're going to use a model called the [Rapid Radiative Transfer Model](http://rtweb.aer.com/rrtm_frame.html) or RRTMG. This is a "serious" and widely-used radiation model, used in many comprehensive GCMs and Numerical Weather Prediction models, that accounts for the wavelength-dependent absorption properties of different gases.

The Python package climlab provides an easy-to-use Python wrapper for the RRTMG code.

#### Creating the model instance

In [ ]:
import climlab

## Do a manual import of vertical levels and water vapor from CESM simulations
## See https://brian-rose.github.io/ClimateLaboratoryBook/courseware/radeq.html#water-vapor-data
lev_cesm = np.array([  3.544638,   7.388814,  13.967214,  23.944625,  37.23029 ,  53.114605,
        70.05915 ,  85.439115, 100.514695, 118.250335, 139.115395, 163.66207 ,
       192.539935, 226.513265, 266.481155, 313.501265, 368.81798 , 433.895225,
       510.455255, 600.5242  , 696.79629 , 787.70206 , 867.16076 , 929.648875,
       970.55483 , 992.5561  ])

qglobal_spec_humid = np.array([2.16104904e-06, 2.15879387e-06, 2.15121262e-06, 2.13630949e-06,
       2.12163684e-06, 2.11168002e-06, 2.09396914e-06, 2.10589390e-06,
       2.42166155e-06, 3.12595653e-06, 5.01369691e-06, 9.60746488e-06,
       2.08907654e-05, 4.78823747e-05, 1.05492451e-04, 2.11889055e-04,
       3.94176751e-04, 7.10734458e-04, 1.34192099e-03, 2.05153261e-03,
       3.16844784e-03, 4.96883408e-03, 6.62218037e-03, 8.38350326e-03,
       9.38620899e-03, 9.65030544e-03])

In [ ]:
#  Make a model on same vertical domain as CESM
mystate = climlab.column_state(lev=lev_cesm, water_depth=2.5)
mystate

In [ ]:
## Give it a radiation model
radmodel = climlab.radiation.RRTMG(name='Radiation (all gases)',  # give our model a name!
                              state=mystate,   # give our model an initial condition!
                              specific_humidity=qglobal_spec_humid,  # tell the model how much water vapor there is
                              albedo = 0.25,  # this the SURFACE shortwave albedo
                              timestep = climlab.constants.seconds_per_day,  # set the timestep to one day (measured in seconds)
                             )
radmodel

Let's look at what we've put together.  We have an object called `radmodel` that has a vertical structure with temperature profile, water vapor, an albedo, and radiatively active gases.  You can inspect the initial state, where $T_s$ describes the surface temperature and $T_{atm}$ the atmospheric temperatures varying with height. 

In [ ]:
#  Here's the state dictionary we already created:
radmodel.state

In [ ]:
#  Here are the pressure levels in hPa
radmodel.lev

And, very importantly: **`radmodel` has specified concentrations of greenhouse gases**, stored in the attribute `absorber_vmr`.

In [ ]:
radmodel.absorber_vmr

Most are just a single number because they are assumed to be well mixed in the atmosphere.  For example, let's look at CO2 concentration in ppm.

In [ ]:
#  E.g. the CO2 content (a well-mixed gas) in parts per million
radmodel.absorber_vmr['CO2'] * 1E6

##### *Discuss:*
Inspect the greenhouse gases in our model.  
   - a. What greenhouse gases are present?
   - b. What is their concentration?
   - c. Which gases that we've discussed are not included in the `absorber_vmr` set?

In [ ]:
## live code to inspect GHGs here

*...your notes here...*

#### Plot the ozone profile
Recall that we talked about the important vertical structure of ozone: O$_3$ hangs out in the upper atmosphere and shields us from UV up there, but has changing concentrations (and effects) in the lower atmosphere due to pollution.  For this reason, we set up our model with a specific vertical profile of ozone, unlike the greenhouse gases that we assume are well-mixed to a uniform concentration.

Let's make a plot to look at the vertical profile of O$_3$.

First, confirm that we have some data available to plot.  We should see two arrays of the same shape; if not, there is a problem.

In [ ]:
# here is the data we need for the plot, as plain numpy arrays:
print(radmodel.lev)
print(radmodel.absorber_vmr['O3'])

In [ ]:
## make a simple plot
fig, ax = plt.subplots()
ax.plot(1E6*radmodel.absorber_vmr['O3'], radmodel.lev)
ax.invert_yaxis() ## higher pressure closer to surface
ax.set(yscale='log', ## log scale for more accurate spatial view
       ylabel='Pressure (hPa)',
       xlabel='Ozone (ppm)')
ax.grid()

#### Water vapor

The other gas that gets special treatment is water vapor.  You may have noticed it is not included in the `radmodel.absorber_vmr` list.  That's because we need water vapor for other climate processes, like precipitation, that are handled separately in the model.  The radiative transfer model still knows there is water vapor, stored in the form of specific humidity at each pressure level:

In [ ]:
#  specific humidity in kg/kg, on the same pressure axis
radmodel.specific_humidity

---
### 2. Running to equilibrium with `while`

Now we are going to let the model run forward in time with these initial greenhouse gas concentrations.

Here are the initial conditions for temperature at the surface and in the atmosphere:

In [ ]:
radmodel.Ts

In [ ]:
radmodel.Tatm

Now let's take a single time step forward and see what happens.

In [ ]:
radmodel.step_forward()

In [ ]:
radmodel.Ts ## inspect the surface temperature

**Do you suppose this system is in radiative equilibrium?**

The main job of the radiative transfer model, which we will not get into in detail here, is to track the shortwave and longwave fluxes up and down between each model layer, accounting for differences across wavelength.  Think of it like a more sophisticated and automated version of our N-layer model, tracking the beams of radiation from surface to top of atmosphere.

We can use `radmodel` to tell us about the radiative balance at the top of the atmosphere, like so:

In [ ]:
radmodel.ASR - radmodel.OLR

To approach radiative equilibrium, we want to step forward in time until the model is very close to energy balance.

We can use a `while` loop, which will run until the top-of-atmosphere imbalance is less than or equal to 0.01 W/m$^2$:

In [ ]:
while np.abs(radmodel.ASR - radmodel.OLR) > 0.01: ## as long as this difference is notable
    radmodel.step_forward() ## keep updating

When the while loop has completed, we should be close to radiative equilibrium, and the energy imbalance should be small.  Check again:

In [ ]:
#  Check the energy budget again
radmodel.ASR - radmodel.OLR

Note that these model attributes are updated "in place" as the model runs -- they are re-set to their most up-to-date value with every time step.

#### Compare to observations
We will compare the thermal profile created by the radiative model with the observed atmospheric temperature profile.  Here we define a helper function to add a vertical profile from a model onto a skew-T.

In [ ]:
def add_profile(skew_fig, model, linestyle='-', color=None):
    line = skew_fig.plot(model.lev, model.Tatm - climlab.constants.tempCtoK,
             label=model.name, linewidth=2)[0]
    skew_fig.plot(1000, model.Ts - climlab.constants.tempCtoK, 'o', 
              markersize=8, color=line.get_color())
    skew_fig.ax.legend()

In [ ]:
skew = make_skewT()
add_profile(skew, radmodel)
skew.ax.set_title('Pure radiative equilibrium', fontsize=18);

---
### 3. Setting up a radiative-convective model

You saw in the radiative equilibrium profiles above that purely radiative balance did not reproduce the observed temperature profile in Earth's atmosphere.  

*Vertical motion (convection)* is another way heat is redistributed in the atmosphere.  We have not yet talked in detail about how convection works -- stay tuned! -- but we can use climlab to build a model that accounts for convection too.

Climlab gives us an easy way to replicate a model so that we can change one or more features and make a direct comparison.  This tool is called `climlab.process_like()`.  We use it here to copy the radiative model and add convection to it.

In [117]:
newrad = climlab.process_like(radmodel)

#  Now create the convection model
conv = climlab.convection.ConvectiveAdjustment(name='Convection',
                                               state=newrad.state,
                                               adj_lapse_rate=6.5,
                                               timestep=newrad.timestep,
                                              )
#  Here is where we build the model by coupling together the two components - letting them interact
rcm = climlab.couple([newrad, conv], name='Radiative-Convective Model')

Integrate for a few years to approach equilibrium.

In [ ]:
rcm.integrate_years(5)

In [ ]:
## Check it
rcm.ASR - rcm.OLR

In [ ]:
skew = make_skewT()
for model in [radmodel, rcm]:
    add_profile(skew, model)

The skew-T plot above shows that the radiative-convective model does indeed get closer to the observed temperature profile.

---
### 4. Model clones
Models are for experimenting and playing with!

We have just built a single-column radiative-convective model with several different absorbing gases. *We can learn about their effects by changing their concentration.*  We'll look at water vapor here.

We will use `process_like()` again to clone our model, this time with the intention to modify the water vapor.

In [ ]:
# Make an exact clone with same temperatures
rcm_noH2O = climlab.process_like(rcm)
rcm_noH2O.name = 'Radiative-Convective Model (No H2O)'

Check the specific humidity:

In [ ]:
#  Check to see that we indeed have the same H2O
rcm_noH2O.specific_humidity == rcm.specific_humidity

Now remove it!

In [ ]:
rcm_noH2O.specific_humidity *= 0

#### Find the radiative forcing from the model diagnostics

The simplest measure of radiative forcing is the instantaneous change in the energy budget before the temperatures have a chance to adjust.  

The model is keeping track of the energy budget for us. To get this we need to call the `compute_diagnostics()` method (but no forward timestep).

In [ ]:
rcm_noH2O.compute_diagnostics()

Now take a look at the changes in the SW and LW budgets between the original and the 2xCO2 case:

In [ ]:
rcm_noH2O.ASR - rcm.ASR

In [ ]:
rcm_noH2O.OLR - rcm.OLR

And we can find the **instantaneous radiative forcing** of removing water vapor, before any temperature response:

In [ ]:
DeltaR_instant = (rcm_noH2O.ASR - rcm_noH2O.OLR) - (rcm.ASR - rcm.OLR)
DeltaR_instant

We run to equilibrium and examine what the temperature profile would look like in this case.

In [ ]:
while abs(rcm_noH2O.ASR - rcm_noH2O.OLR) > 0.01:
    rcm_noH2O.step_forward()

In [ ]:
skew = make_skewT()
for model in [rcm, rcm_noH2O]:
    add_profile(skew, model)

We're going to use a model called the [Rapid Radiative Transfer Model](http://rtweb.aer.com/rrtm_frame.html) or RRTMG. This is a "serious" and widely-used radiation model, used in many comprehensive GCMs and Numerical Weather Prediction models.

climlab provides an easy-to-use Python wrapper for the RRTMG code.

### Creating the model instance

In [ ]:
import climlab

## Do a hacky import of vertical levels and water vapor from CESM simulations
## See https://brian-rose.github.io/ClimateLaboratoryBook/courseware/radeq.html#water-vapor-data
lev_cesm = np.array([  3.544638,   7.388814,  13.967214,  23.944625,  37.23029 ,  53.114605,
        70.05915 ,  85.439115, 100.514695, 118.250335, 139.115395, 163.66207 ,
       192.539935, 226.513265, 266.481155, 313.501265, 368.81798 , 433.895225,
       510.455255, 600.5242  , 696.79629 , 787.70206 , 867.16076 , 929.648875,
       970.55483 , 992.5561  ])

qglobal_spec_humid = np.array([2.16104904e-06, 2.15879387e-06, 2.15121262e-06, 2.13630949e-06,
       2.12163684e-06, 2.11168002e-06, 2.09396914e-06, 2.10589390e-06,
       2.42166155e-06, 3.12595653e-06, 5.01369691e-06, 9.60746488e-06,
       2.08907654e-05, 4.78823747e-05, 1.05492451e-04, 2.11889055e-04,
       3.94176751e-04, 7.10734458e-04, 1.34192099e-03, 2.05153261e-03,
       3.16844784e-03, 4.96883408e-03, 6.62218037e-03, 8.38350326e-03,
       9.38620899e-03, 9.65030544e-03])

In [ ]:
#  Make a model on same vertical domain as CESM
mystate = climlab.column_state(lev=lev_cesm, water_depth=2.5)
mystate

In [ ]:
## Give it a radiation model
radmodel = climlab.radiation.RRTMG(name='Radiation (all gases)',  # give our model a name!
                              state=mystate,   # give our model an initial condition!
                              specific_humidity=qglobal_spec_humid,  # tell the model how much water vapor there is
                              albedo = 0.25,  # this the SURFACE shortwave albedo
                              timestep = climlab.constants.seconds_per_day,  # set the timestep to one day (measured in seconds)
                             )
radmodel

Let's look at what we've put together.  We have an object called `radmodel` that has a vertical structure with temperature profile, water vapor, an albedo, and radiatively active gases.  You can inspect the initial state: $T_s$ for the surface temperature and $T_{atm}$ for atmospheric temperatures varying with height. 

In [ ]:
#  Here's the state dictionary we already created:
radmodel.state

In [ ]:
#  Here are the pressure levels in hPa
radmodel.lev

And, very importantly: **`radmodel` has specified concentrations of greenhouse gases**, stored in the attribute `absorber_vmr`

In [ ]:
radmodel.absorber_vmr

Most are just a single number because they are assumed to be well mixed in the atmosphere.  For example, let's look at CO2 concentration in ppm.

In [ ]:
#  E.g. the CO2 content (a well-mixed gas) in parts per million
radmodel.absorber_vmr['CO2'] * 1E6

---
### Double CO2 concentration in the radiative model

---
### Add convection

---
## Part 2: Lab procedure

1. Set up a **radiative-convective** model instance called `my_rcm`.  Assign it the same vertical levels, initial specific humidity, climate state, and convective module as the base example.  Run it forward for 5 years.  Check whether it is close to energy balance.
2. Use `climlab.process_like(...)` to make a model clone of `my_rcm`.  Set the `.name` attribute of the model clone to 'Radiation (Double CO2)' and double its CO$_2$ concentration
    - Note: the CO2 concentration is in the radiation module of your model instance, which you access using `model_name.subprocess[name_of_subprocess]`
3. Use `compute_diagnostics` to compute the *instantaneous radiative forcing* of this change in the model.
4. Run the model forward to equilibrium.  Compare its vertical temperature profile with `my_rcm` and with observations on a skew-T diagram.
    - Interpret: What happens to the temperature in the lower atmosphere as a result of doubling CO$_2$?  What about the upper atmosphere?
5. Describe how you would use the tools from this lab to investigate the role of convection in shaping the atmospheric response to greenhouse gases.  Time allowing: try it!
    

In [ ]:
## add code and markdown cells here to complete the lab

---
## Endnotes
- This [National Weather Service interactive](https://www.weather.gov/jetstream/skewt) will help orient you to the lines included on a skew-T diagram.  Focus on the temperature and pressure lines.  In the atmosphere, pressure is a proxy for elevation.
- In the online climlab documentation, you can read about the two different "GCM-level" radiation codes provided with climlab:
    - The [RRTMG (Rapid Radiative Transfer Model)](https://climlab.readthedocs.io/en/latest/api/climlab.radiation.RRTMG.html) which is used in many current GCMs.
    - The [CAM3 radiation module](https://climlab.readthedocs.io/en/latest/api/climlab.radiation.CAM3.html) from NCAR (essentially the same radiation code used in the Community Earth System Model)
    
- Read more about *while*-loops in [Python for Everyone, Chapter 5](https://www.py4e.com/html3/05-iterations).

- This lab is based on chapters 11-13 of [The Climate Laboratory](https://brian-rose.github.io/ClimateLaboratoryBook) by [Brian E. J. Rose](http://www.atmos.albany.edu/facstaff/brose/index.html), University at Albany.

- Lab notebook last updated by Lizz Ultee, 12 Mar 2024